In [55]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

In [ ]:
base_url = 'https://api.mobygames.com/v1/'
endpoint_var = 'games'
api_key = open("../data/mobygames_api_key.txt")
endpoint= base_url + endpoint_var

In [ ]:
parameters = {"format":"normal", 
              "api_key": api_key, 
              "genre":158,
              "offset":99
#               "id":2
             }

In [ ]:
response = requests.get(endpoint, params=parameters)
response

In [ ]:
res = response.json()

In [ ]:
type(res)

In [ ]:
len(res["games"])

In [ ]:
res["games"][0].keys()

In [ ]:
print(res["games"][0]["moby_url"])

In [ ]:
#this returns one of the first release dates from one of the platforms - this is one way to get dates
print(res["games"][0]["platforms"][0]["first_release_date"])
print(res["games"][0]["platforms"][0]["platform_id"])

In [ ]:
# this gets to a json file that actually has the companies and their roles 
# https://api.mobygames.com/v1/games/2/platforms/2?api_key=MYKEY

### A function for looping through responses of 100, offset by 100, until it hits the end
the goal of this funciton is to fill up the data dictionary with a bunch of other response dictionaries that contain all games for the given "genre" - in this case, 158, or "vehicle simulator"

In [19]:
y=100
x=1
data = {}

In [20]:
# WARNING: THIS DOESN'T BREAK - RESPONSE WILL BE 200 EVEN IF OFFSET IS TOO HIGH, APPARENTLY - 
# NEED TO CHECK THE RES["GAMES"] DICT TO SEE IF EMPTY, THATS THE WAY TO DO IT - use len like this:
# len(data["games12"])

while True:
        base_url = 'https://api.mobygames.com/v1/'
        endpoint_var = 'games'
        api_key = open("../data/mobygames_api_key.txt")
        endpoint= base_url + endpoint_var
        parameters = {"format":"normal", 
                  "api_key": api_key, 
                  "genre":158,
                  "offset":y}
        response = requests.get(endpoint, params=parameters)
        print(response)
        res = response.json()
        print(len(res["games"]))
        if len(res["games"]) > 0:
            res["games"+str(x)] = res["games"]
            del res["games"]
            data = {**data, **res}
            y+=100
            x+=1
            sleep(10.1)
        else:
            break

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [21]:
len(data.keys())

12

In [27]:
import pickle
f = open("../data/vehicle_simulator_genre_158.pkl","wb")
pickle.dump(data,f)
f.close()


In [24]:
len(data["games12"])

50

In [26]:
data.keys()

dict_keys(['games1', 'games2', 'games3', 'games4', 'games5', 'games6', 'games7', 'games8', 'games9', 'games10', 'games11', 'games12'])

### create a function that takes the data variable and outputs everything into a dataframe

In [138]:
title = []
game_id = []
description = []
moby_url = []
official_url = []
first_release_date = []
first_platform_id = []
first_platform_name = []
second_release_date = []
second_platform_id = []
second_platform_name = []
platform_dict = {}


In [139]:
for key, values_list in data.items():
    for vals_dicts in values_list:
        for headers, values in vals_dicts.items():
            if headers == "platforms":
                num_platforms = len(values)
                range_num = list(range(0,num_platforms))
                #dictionary might not do what I need it to do here - need to think harder about these lists
                # one problem: lists need to be outside the for loop below!! I thknk that would fix prob
                release_date = []
                platform_id = []
                platform_name = []
                for num in range_num:
                    release_date_key = "release_date" + str(num)
#                     release_date = [] 
                    release_date.append(values[num]["first_release_date"])
                    platform_id_key = "platform_id" + str(num)
#                     platform_id = []
                    platform_id.append(values[num]["platform_id"])
                    platform_name_key = "platform_name" + str(num)
#                     platform_name = []
                    platform_name.append(values[num]["platform_name"])
                    platform_dict[release_date_key] = release_date
                    platform_dict[platform_id_key] = platform_id
                    platform_dict[platform_name_key] = platform_name
                    
                    
#                 first_release_date.append(values[0]["first_release_date"])
#                 first_platform_id.append(values[0]["platform_id"])
#                 first_platform_name.append(values[0]["platform_name"])
#                 try:
#                     second_release_date.append(values[1]["first_release_date"])
#                     second_platform_id.append(values[1]["platform_id"])
#                     second_platform_name.append(values[1]["platform_name"])
#                 except:
#                     pass
            elif headers == "description":
                description.append(values)
            elif headers == "title":
                title.append(values)
            elif headers == "moby_url":
                moby_url.append(values)
            elif headers == "game_id":
                game_id.append(values)
            elif headers == "official_url":
                official_url.append(values)

In [140]:
main_list = []
main_list.append(title)
main_list.append(game_id)
main_list.append(description)
main_list.append(moby_url)
main_list.append(official_url)
main_list.append(first_release_date)
main_list.append(first_platform_id)
main_list.append(first_platform_name)
main_list.append(second_release_date)
main_list.append(second_platform_id)
main_list.append(second_platform_name)

vehicle_sim_158 = pd.DataFrame(main_list).transpose()
vehicle_sim_158.columns = ["title", "game_id", "description", "moby_url", "official_url", "first_release_date", "first_platform_id", "first_platform_name", "second_release_date", "second_platform_id", "second_platform_name"]


In [141]:
vehicle_sim_158[100:149]

,title,game_id,description,moby_url,official_url,first_release_date,first_platform_id,first_platform_name,second_release_date,second_platform_id,second_platform_name
100,Colin McRae Rally 3,8684,The third title in the rally series has you of...,http://www.mobygames.com/game/colin-mcrae-rally-3,None,None,None,None,None,None,None
101,MotoGP 3,8700,Take to the track on the greatest 2-stroke and...,http://www.mobygames.com/game/motogp-3,None,None,None,None,None,None,None
102,Sega GT 2002,8719,"Race everything from street-legal 60hp $6,000 ...",http://www.mobygames.com/game/sega-gt-2002,None,None,None,None,None,None,None
103,Pro Race Driver,8767,<i>Race Driver</i> adds a storyline and career...,http://www.mobygames.com/game/pro-race-driver,None,None,None,None,None,None,None
104,Porsche Challenge,8854,Porsche Challenge is a racing game exclusively...,http://www.mobygames.com/game/porsche-challenge,None,None,None,None,None,None,None
105,NASCAR: Dirt to Daytona,8892,NASCAR action with 4 different series: Weekly ...,http://www.mobygames.com/game/nascar-dirt-to-d...,None,None,None,None,None,None,None
106,Crazy Taxi: Catch a Ride,8894,Crazy Taxi jumps into the handheld world head ...,http://www.mobygames.com/game/crazy-taxi-catch...,None,None,None,None,None,None,None
107,CART Precision Racing,8955,CART Precision Racing is american racing leagu...,http://www.mobygames.com/game/cart-precision-r...,None,None,None,None,None,None,None
108,Big Scale Racing,8963,In <i>Big Scale Racing</i> you can race with 1...,http://www.mobygames.com/game/big-scale-racing,None,None,None,None,None,None,None
109,Big Mutha Truckers,8964,"Ma Jackson, the owner of Big Mutha Truckers, t...",http://www.mobygames.com/game/big-mutha-truckers,None,None,None,None,None,None,None


In [144]:
platform_dict

{'release_date0': ['2021-07-07'],
 'platform_id0': [3],
 'platform_name0': ['Windows'],
 'release_date1': ['2021-11-26'],
 'platform_id1': [288],
 'platform_name1': ['PlayStation 5'],
 'release_date2': ['2017-08-31'],
 'platform_id2': [91],
 'platform_name2': ['Android'],
 'release_date3': ['2021-11-19'],
 'platform_id3': [203],
 'platform_name3': ['Nintendo Switch'],
 'release_date4': ['2021-11-22'],
 'platform_id4': [141],
 'platform_name4': ['PlayStation 4'],
 'release_date5': ['2021-11-22'],
 'platform_id5': [281],
 'platform_name5': ['Stadia'],
 'release_date6': ['2021-11-22'],
 'platform_id6': [288],
 'platform_name6': ['PlayStation 5'],
 'release_date7': ['2021-03-17'],
 'platform_id7': [289],
 'platform_name7': ['Xbox Series'],
 'release_date8': ['2015-10-30'],
 'platform_id8': [179],
 'platform_name8': ['tvOS'],
 'release_date9': ['2017-09-14'],
 'platform_id9': [203],
 'platform_name9': ['Nintendo Switch'],
 'release_date10': ['1983'],
 'platform_id10': [149],
 'platform_name